In [ ]:
import aliases

In [ ]:
import itertools as _itertools
from collections import deque as _deque
import sys as _sys

import h5py as _h5py

In [ ]:
'1'.zfill(2)

In [ ]:
def message(i, maxi, name = ''):
    frac = i / maxi
    msg = ''.join((
        "\r",
        f" {str(i).rjust(9)} / {str(maxi).ljust(9)} ",
        f" {name} ".ljust(32),
        f" [{(round(25 * frac) * '#').ljust(25, '.')}] ",
        f" ({str(round(100 * frac)).zfill(2)}%) ",
        )).ljust(79)
    if frac == 1:
        msg += '\n'
    _sys.stdout.write(msg)
    _sys.stdout.flush()

def record_manifest(h5obj, manifest = None):
    if mannone := (manifest is None):
        manifest = dict()
    length = len(h5obj)
    message(0, length)
    for i, name in enumerate(h5obj):
        try:
            record_manifest_sub(
                h5obj[name],
                manifest.setdefault(name, _deque()),
                )
            message(i, length, name)
        except KeyError:
            continue
    message(length, length, 'Complete.')
    if mannone:
        return manifest
        
def record_manifest_sub(h5obj, manifest, prename = ''):
    manifest.extend((f"{prename}.{name}" for name in h5obj.attrs))
    if isinstance(h5obj, _h5py.Group):
        for name in h5obj:
            try:
                record_manifest_sub(
                    h5obj[name],
                    manifest,
                    f"{prename}/{name}"
                    )
            except KeyError:
                continue
    else:
        manifest.append(f"{prename}#{h5obj.shape}".replace(' ', ''))

In [ ]:
%%time
with _h5py.File('../data/allout.frm', mode = 'r') as h5file:
    out = record_manifest(h5file)

In [ ]:
import itertools

In [ ]:
%%time
filt = tuple(filter(lambda x: x.endswith('d'), itertools.chain.from_iterable(out.values())))

In [ ]:
filt